In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load data from CSV files
food_categories = pd.read_csv("/Users/pratiksha/Downloads/food_categories.csv")
food_preferences = pd.read_csv("/Users/pratiksha/Downloads/food_preferences.csv")
nutrient_content = pd.read_csv("/Users/pratiksha/Downloads/nutrient_content.csv", index_col=0)
nutrient_requirements = pd.read_csv("/Users/pratiksha/Downloads/nutrient_requirements.csv", index_col=0)

# Create a new model
model = gp.Model("Diet Optimization")

# Decision variables
foods = range(len(food_categories))
weeks = range(1)  # Assuming optimization for one week only

x = model.addVars(foods, weeks, vtype=GRB.CONTINUOUS, name="food")

# Objective function
model.setObjective(sum(x[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)

# Nutritional balance constraints
for j in weeks:
    for k in nutrient_requirements.index:
        model.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x[i, j] for i in foods) >= nutrient_requirements.loc[k, "Min_Requirement"])
        model.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x[i, j] for i in foods) <= nutrient_requirements.loc[k, "Max_Requirement"])

# Dietary preferences constraints
for i in foods:
    model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["All_grams"])
    if food_categories.iloc[i]["Is_Vegetarian"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Veggie_grams"])
    if food_categories.iloc[i]["Is_Vegan"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Vegan_grams"])
    if food_categories.iloc[i]["Is_Kosher"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Kosher_grams"])
    if food_categories.iloc[i]["Is_Halal"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Halal_grams"])

# Variety constraints
for i in foods:
    model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= 0.03 * sum(food_preferences.iloc[0][["Veggie_grams", "Vegan_grams", "Kosher_grams", "Halal_grams", "All_grams"]]))

# Solve the model
model.optimize()

# Print the optimal diet plan
for i in foods:
    for j in weeks:
        if x[i, j].x > 0:
            print(f"Food item {food_categories.iloc[i]['Food_Item']} for week {j}: {x[i, j].x} grams")

# Print the total cost
print(f"Total cost: {model.objVal}")


Restricted license - for non-production use only - expires 2025-11-24


/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_38330/2184639533.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 528 rows, 120 columns and 14808 nonzeros
Model fingerprint: 0x2dbc4a56
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+05]
Presolve removed 468 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 180 columns, 7260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.370451e+04   0.000000e+00      0s
      20    6.0217307e+04   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.021730696e+04
Food item Food_4 for week 0: 10711.51104427281 grams
Food item Food_6 for week 0: 8880.744112402806 grams
Food item Food_15 for week 0: 17310.0 grams
Food item Food_16 f

In [2]:
# After adding all constraints to the model
total_constraints = model.NumConstrs
print(f"Total number of constraints (excluding non-negativity): {total_constraints}")


Total number of constraints (excluding non-negativity): 528


In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

#Load the dataset
food_categories = pd.read_csv("/Users/pratiksha/Downloads/food_categories.csv")
food_preferences = pd.read_csv("/Users/pratiksha/Downloads/food_preferences.csv")
nutrient_content = pd.read_csv("/Users/pratiksha/Downloads/nutrient_content.csv", index_col=0)
nutrient_requirements = pd.read_csv("/Users/pratiksha/Downloads/nutrient_requirements.csv", index_col=0)

# Create a new model
model = gp.Model("Diet Optimization")

# Decision variables
foods = range(len(food_categories))
weeks = range(1)  # Assuming optimization for one week only

x = model.addVars(foods, weeks, vtype=GRB.CONTINUOUS, name="food")

# Objective function
model.setObjective(sum(x[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)

# Nutritional balance constraints
for j in weeks:
    for k in nutrient_requirements.index:
        model.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x[i, j] for i in foods) >= nutrient_requirements.loc[k, "Min_Requirement"])
        model.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x[i, j] for i in foods) <= nutrient_requirements.loc[k, "Max_Requirement"])

# Dietary preferences constraints
for i in foods:
    model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["All_grams"])
    if food_categories.iloc[i]["Is_Vegetarian"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Veggie_grams"])
    if food_categories.iloc[i]["Is_Vegan"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Vegan_grams"])
    if food_categories.iloc[i]["Is_Kosher"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Kosher_grams"])
    if food_categories.iloc[i]["Is_Halal"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Halal_grams"])

# Variety constraints
for i in foods:
    model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= 0.03 * sum(food_preferences.iloc[0][["Veggie_grams", "Vegan_grams", "Kosher_grams", "Halal_grams", "All_grams"]]))

# Solve the model
model.optimize()

# Print the optimal food production cost
print(f"Optimal food production cost: ${model.objVal:.2f}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 528 rows, 120 columns and 14808 nonzeros
Model fingerprint: 0x2dbc4a56
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+05]
Presolve removed 468 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 180 columns, 7260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.370451e+04   0.000000e+00      0s
      20    6.0217307e+04   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.021730696e+04
Optimal food production cost: $60217.31


In [8]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


#Load the dataset
food_categories = pd.read_csv("/Users/pratiksha/Downloads/food_categories.csv")
food_preferences = pd.read_csv("/Users/pratiksha/Downloads/food_preferences.csv")
nutrient_content = pd.read_csv("/Users/pratiksha/Downloads/nutrient_content.csv", index_col=0)
nutrient_requirements = pd.read_csv("/Users/pratiksha/Downloads/nutrient_requirements.csv", index_col=0)

# Create a new model
model = gp.Model("Diet Optimization")

# Decision variables
foods = range(len(food_categories))
weeks = range(1)  # Assuming optimization for one week only

x = model.addVars(foods, weeks, vtype=GRB.CONTINUOUS, name="food")

# Objective function
model.setObjective(sum(x[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)

# Nutritional balance constraints
for j in weeks:
    for k in nutrient_requirements.index:
        model.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x[i, j] for i in foods) >= nutrient_requirements.loc[k, "Min_Requirement"])
        model.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x[i, j] for i in foods) <= nutrient_requirements.loc[k, "Max_Requirement"])

# Dietary preferences constraints
for i in foods:
    model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["All_grams"])
    if food_categories.iloc[i]["Is_Vegetarian"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Veggie_grams"])
    if food_categories.iloc[i]["Is_Vegan"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Vegan_grams"])
    if food_categories.iloc[i]["Is_Kosher"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Kosher_grams"])
    if food_categories.iloc[i]["Is_Halal"]:
        model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= food_preferences.iloc[0]["Halal_grams"])

# Variety constraints
for i in foods:
    model.addConstr(gp.quicksum(x[i, j] for j in weeks) <= 0.03 * sum(food_preferences.iloc[0][["Veggie_grams", "Vegan_grams", "Kosher_grams", "Halal_grams", "All_grams"]]))

# Solve the model
model.optimize()

# Print the optimal food production cost
print(f"Optimal food production cost: ${model.objVal:.2f}")

# Extract solution
solution = {food_categories.iloc[i]["Food_Item"]: x[i, 0].x for i in foods}

# Calculate total grams from Halal and Kosher foods
halal_grams = sum([solution.get("Food_15", 0), solution.get("Food_42", 0), solution.get("Food_48", 0), solution.get("Food_50", 0), solution.get("Food_62", 0)])
kosher_grams = sum([solution.get("Food_16", 0), solution.get("Food_44", 0), solution.get("Food_47", 0), solution.get("Food_49", 0), solution.get("Food_55", 0), solution.get("Food_58", 0), solution.get("Food_61", 0), solution.get("Food_70", 0), solution.get("Food_80", 0)])


# Calculate proportions
total_grams = sum(solution.values())
halal_proportion = halal_grams / total_grams
kosher_proportion = kosher_grams / total_grams

# Print proportions
print(f"Proportion of grams from Halal foods: {halal_proportion:.4f}")
print(f"Proportion of grams from Kosher foods: {kosher_proportion:.4f}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 528 rows, 120 columns and 14808 nonzeros
Model fingerprint: 0x2dbc4a56
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+05]
Presolve removed 468 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 180 columns, 7260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.370451e+04   0.000000e+00      0s
      20    6.0217307e+04   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.021730696e+04
Optimal food production cost: $60217.31
Proportion of grams from Halal foods: 0.1442
Proportion of grams from Kosher foods: 0.2426


In [9]:
# Create a new model without the Variety constraint(s)
model_without_variety = gp.Model("Diet Optimization Without Variety Constraint")

# Decision variables
x_without_variety = model_without_variety.addVars(foods, weeks, vtype=GRB.CONTINUOUS, name="food")

# Objective function
model_without_variety.setObjective(sum(x_without_variety[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)

# Nutritional balance constraints
for j in weeks:
    for k in nutrient_requirements.index:
        model_without_variety.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x_without_variety[i, j] for i in foods) >= nutrient_requirements.loc[k, "Min_Requirement"])
        model_without_variety.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x_without_variety[i, j] for i in foods) <= nutrient_requirements.loc[k, "Max_Requirement"])

# Dietary preferences constraints
for i in foods:
    model_without_variety.addConstr(gp.quicksum(x_without_variety[i, j] for j in weeks) <= food_preferences.iloc[0]["All_grams"])
    if food_categories.iloc[i]["Is_Vegetarian"]:
        model_without_variety.addConstr(gp.quicksum(x_without_variety[i, j] for j in weeks) <= food_preferences.iloc[0]["Veggie_grams"])
    if food_categories.iloc[i]["Is_Vegan"]:
        model_without_variety.addConstr(gp.quicksum(x_without_variety[i, j] for j in weeks) <= food_preferences.iloc[0]["Vegan_grams"])
    if food_categories.iloc[i]["Is_Kosher"]:
        model_without_variety.addConstr(gp.quicksum(x_without_variety[i, j] for j in weeks) <= food_preferences.iloc[0]["Kosher_grams"])
    if food_categories.iloc[i]["Is_Halal"]:
        model_without_variety.addConstr(gp.quicksum(x_without_variety[i, j] for j in weeks) <= food_preferences.iloc[0]["Halal_grams"])

# Solve the model without the Variety constraint(s)
model_without_variety.optimize()

# Extract solution
solution_without_variety = {food_categories.iloc[i]["Food_Item"]: x_without_variety[i, 0].x for i in foods}

# Count the number of food items produced
num_food_items_original = sum(1 for val in solution.values() if val > 0)
num_food_items_without_variety = sum(1 for val in solution_without_variety.values() if val > 0)
num_food_items_difference = num_food_items_original - num_food_items_without_variety

# Calculate the production cost for both models
production_cost_original = model.objVal
production_cost_without_variety = model_without_variety.objVal
production_cost_difference = production_cost_original - production_cost_without_variety

# Print results
print(f"Number of food items produced without the Variety constraint(s): {num_food_items_without_variety}")
print(f"Number of fewer food items produced: {num_food_items_difference}")
print(f"Production cost without the Variety constraint(s): ${production_cost_without_variety:.2f}")
print(f"Difference in production cost: ${production_cost_difference:.2f}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 408 rows, 120 columns and 14688 nonzeros
Model fingerprint: 0xb15d3781
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+05]
Presolve removed 348 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 180 columns, 7260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.370451e+04   0.000000e+00      0s
      13    5.1419739e+04   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.141973922e+04
Number of food items produced without the Variety constraint(s): 26
Number of fewer food items produced: 21
Production cost without the Variety constraint(s): $51419.74


In [10]:
# Update the right-hand-side values of the dietary preference constraints
food_preferences_corrected = food_preferences.copy()
food_preferences_corrected.iloc[0] += 10000

# Create a new model with the corrected dietary preference constraints
model_corrected = gp.Model("Diet Optimization Corrected")

# Decision variables
x_corrected = model_corrected.addVars(foods, weeks, vtype=GRB.CONTINUOUS, name="food")

# Objective function
model_corrected.setObjective(sum(x_corrected[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)

# Nutritional balance constraints
for j in weeks:
    for k in nutrient_requirements.index:
        model_corrected.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x_corrected[i, j] for i in foods) >= nutrient_requirements.loc[k, "Min_Requirement"])
        model_corrected.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x_corrected[i, j] for i in foods) <= nutrient_requirements.loc[k, "Max_Requirement"])

# Corrected dietary preferences constraints
for i in foods:
    model_corrected.addConstr(gp.quicksum(x_corrected[i, j] for j in weeks) <= food_preferences_corrected.iloc[0]["All_grams"])
    if food_categories.iloc[i]["Is_Vegetarian"]:
        model_corrected.addConstr(gp.quicksum(x_corrected[i, j] for j in weeks) <= food_preferences_corrected.iloc[0]["Veggie_grams"])
    if food_categories.iloc[i]["Is_Vegan"]:
        model_corrected.addConstr(gp.quicksum(x_corrected[i, j] for j in weeks) <= food_preferences_corrected.iloc[0]["Vegan_grams"])
    if food_categories.iloc[i]["Is_Kosher"]:
        model_corrected.addConstr(gp.quicksum(x_corrected[i, j] for j in weeks) <= food_preferences_corrected.iloc[0]["Kosher_grams"])
    if food_categories.iloc[i]["Is_Halal"]:
        model_corrected.addConstr(gp.quicksum(x_corrected[i, j] for j in weeks) <= food_preferences_corrected.iloc[0]["Halal_grams"])

# Solve the corrected model
model_corrected.optimize()

# Compare the objective function values
original_objective = model.objVal
corrected_objective = model_corrected.objVal
difference = corrected_objective - original_objective

# Print the results
print(f"Original Objective Function Value: {original_objective}")
print(f"Corrected Objective Function Value: {corrected_objective}")
print(f"Difference in Objective Function Value: {difference}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 408 rows, 120 columns and 14688 nonzeros
Model fingerprint: 0x3c9f1352
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+05]
Presolve removed 348 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 180 columns, 7260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.370451e+04   0.000000e+00      0s
      22    4.9168726e+04   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.916872583e+04
Original Objective Function Value: 60217.30695597733
Corrected Objective Function Value: 49168.72583208337
Difference in Objective Function Value: -11048.581123893957


In [11]:
# Create a new model for sensitivity analysis
model_sensitivity = gp.Model("Diet Optimization Sensitivity")

# Decision variables
x_sensitivity = model_sensitivity.addVars(foods, weeks, vtype=GRB.CONTINUOUS, name="food")

# Objective function
model_sensitivity.setObjective(sum(x_sensitivity[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)

# Nutritional balance constraints
for j in weeks:
    for k in nutrient_requirements.index:
        model_sensitivity.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x_sensitivity[i, j] for i in foods) >= nutrient_requirements.loc[k, "Min_Requirement"])
        model_sensitivity.addConstr(gp.quicksum(nutrient_content.loc[food_categories.iloc[i]["Food_Item"], k] * x_sensitivity[i, j] for i in foods) <= nutrient_requirements.loc[k, "Max_Requirement"])

# Dietary preferences constraints
for i in foods:
    model_sensitivity.addConstr(gp.quicksum(x_sensitivity[i, j] for j in weeks) <= food_preferences.iloc[0]["All_grams"])
    if food_categories.iloc[i]["Is_Vegetarian"]:
        model_sensitivity.addConstr(gp.quicksum(x_sensitivity[i, j] for j in weeks) <= food_preferences.iloc[0]["Veggie_grams"])
    if food_categories.iloc[i]["Is_Vegan"]:
        model_sensitivity.addConstr(gp.quicksum(x_sensitivity[i, j] for j in weeks) <= food_preferences.iloc[0]["Vegan_grams"])
    if food_categories.iloc[i]["Is_Kosher"]:
        model_sensitivity.addConstr(gp.quicksum(x_sensitivity[i, j] for j in weeks) <= food_preferences.iloc[0]["Kosher_grams"])
    if food_categories.iloc[i]["Is_Halal"]:
        model_sensitivity.addConstr(gp.quicksum(x_sensitivity[i, j] for j in weeks) <= food_preferences.iloc[0]["Halal_grams"])

# Add constraint to include the first food item
model_sensitivity.addConstr(gp.quicksum(x_sensitivity[0, j] for j in weeks) >= 1)

# Solve the sensitivity analysis model for different costs of the first food item
cost_reduction = 0
while True:
    # Set new cost for the first food item
    food_categories.at[0, "Cost_per_gram"] -= 0.001
    # Update the objective function with the new cost
    model_sensitivity.setObjective(sum(x_sensitivity[i, j] * food_categories.iloc[i]["Cost_per_gram"] for i in foods for j in weeks), GRB.MINIMIZE)
    # Solve the model
    model_sensitivity.optimize()
    # Check if the first food item is included in the optimal solution
    if x_sensitivity[0, 0].x > 0.5:
        break
    cost_reduction += 0.001

# Print the results
print(f"Amount the first food item needs to be less costly: {cost_reduction}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 409 rows, 120 columns and 14689 nonzeros
Model fingerprint: 0x59f0567b
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+05]
Presolve removed 349 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 180 columns, 7260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3132203e-01   2.370447e+04   0.000000e+00      0s
      13    5.1419785e+04   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.141978485e+04
Amount the first food item needs to be less costly: 0
